In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta, date, timezone
from analyzer.live_entry_strats import LiveEntryStrats as les
from analyzer.live_exit_strats import LiveExitStrats as lxs
from time import sleep
import time
from database.comet import Comet
from tqdm import tqdm
from coinbase.coinbase import Coinbase as cbs
from processor.processor import Processor as p
import pytz

In [2]:
comet = Comet()

In [3]:
whitelist_symbols = ['ADA', 'BTC', 'DOGE', 'ETH', 'SHIB', 'WLUNA',
       'AVAX', 'LTC', 'DOT']

In [4]:
retrack_days = 3
req = 0.2
signal = 0.30
value = False
conservative = True
entry_strategy = "all"
exit_strategy = "adaptive_hold"
fee = 0.005
minimum_rows = retrack_days * 3

In [ ]:
##CONSTANTS
count = 0
while count < 6:
    comet.connect()
    end = datetime.now().astimezone(pytz.UTC)
    start = (end - timedelta(days=30)).astimezone(pytz.UTC)
    accounts = cbs.get_accounts()
    balance = accounts[accounts["currency"]=="USD"]["balance"].iloc[0]
    pending_orders = cbs.get_orders()
    pending_orders = p.live_column_date_processing(pending_orders.rename(columns={"created_at":"date"}))
    spots = []
    historicals = []
    for currency in whitelist_symbols:
        try:
            spot = cbs.get_current_price(currency)
            historical = cbs.get_timeframe_prices(currency,end,minimum_rows)
            spot["crypto"] = currency
            if "message" not in list(spot.keys()):
                spots.append(spot)
            historicals.append(historical)
        except:
            print(currency)
            continue
    current_spots = pd.DataFrame(spots)
    current_historicals = pd.concat(historicals)
    current_historicals.sort_values("date",inplace=True)
    ns = []
    for crypto in whitelist_symbols:
        try:
            crypto_sim = current_historicals[current_historicals["crypto"]==crypto].copy()
            crypto_sim.sort_values("date",inplace=True)
            crypto_sim["signal"] = crypto_sim["close"].pct_change(retrack_days)
            crypto_sim["velocity"] = crypto_sim["signal"].pct_change()
            crypto_sim["inflection"] = crypto_sim["velocity"].pct_change()
            crypto_sim["p_sign_change"] = [row[1]["velocity"] * row[1]["inflection"] < 0 for row in crypto_sim.iterrows()]
            ns.append(crypto_sim.iloc[-1])
        except Exception as e:
            print(crypto,str(e))
    final = pd.DataFrame(ns)
    merged = final.merge(current_spots.drop("volume",axis=1),on="crypto")
    comet.store("coinbase_hourly",merged)
    fls = []
    for currency in accounts["currency"].unique():
        fill = cbs.get_fill(currency)
        if len(fill) > 0:
            try:
                f = pd.DataFrame(fill)
                fls.append(f)
            except:
                continue
    ## sells
    if len(fls) > 0:
        fills = pd.concat(fls)
        #store_non_existing_executed_buy
        ##document_buys
        existing_fills = comet.retrieve_fills()
        if existing_fills.index.size > 1:
            new_fills = fills[~fills["order_id"].isin(list(existing_fills["order_id"]))]
            if new_fills.index.size > 1:
                incomplete_trades = new_fills[(new_fills["side"]=="buy")]
                incomplete_trades["size"] = [float(x) for x in incomplete_trades["size"]]
                incomplete_trades["price"] = [float(x) for x in incomplete_trades["price"]]
                for oi in incomplete_trades["order_id"].unique():
                    order_trades = incomplete_trades[incomplete_trades["order_id"]==oi]
                    if len([x for x in order_trades["settled"] if x == False]) == 0:
                        comet.store("fills",order_trades)
                        size = round(sum(order_trades["size"]),6)
                        buy_price = order_trades["price"].mean()
                        product_id = order_trades.iloc[0]["product_id"]
                        exits = lxs.exit_analysis("adaptive_hold",merged)
                        exits = merged[(merged["crypto"]==product_id.split("-")[0])
                                       & (merged["p_sign_change"]==True)
                                        & (merged["velocity"] <= 3)
                                        & (merged["velocity"] > 0)
                                        & (merged["inflection"] <= 1)
                                        & (merged["inflection"] >= -1)]
                        if exits.index.size > 0:
                            exit = exits.iloc[0]
                            sell_price = float(exit["bid"])
                            incomplete_trade = order_trades.iloc[0]
                            incomplete_trade["size"] = size
                            sell_statement = cbs.place_sell(product_id,sell_price,size)
                            comet.store("orders",pd.DataFrame([sell_statement]))
                            incomplete_trade["sell_id"] = sell_statement["id"]
                            comet.store("incomplete_trades",pd.DataFrame([incomplete_trade]))
                completed_trades = new_fills[(new_fills["side"]=="sell")]
                for soi in completed_trades["order_id"].unique():
                    sell_order_trades = completed_trades[completed_trades["order_id"]==soi]
                    if len([x for x in sell_order_trades["settled"] if x == False]) == 0:
                        comet.store("fills",sell_order_trades)
                        complete_trade = sell_order_trades.iloc[0]
                        order_id = complete_trade["order_id"]
                        one_half = comet.retrieve_incomplete_trade(order_id)
                        one_half["sell_date"] = complete_trade["created_at"]
                        one_half["sell_price"] = complete_trade["price"]
                        comet.store("complete_trades",one_half)
    # ##buys
    if balance > 1:
        offerings = les.entry_analysis("all",merged,signal,value,conservative)
        ##BUYS
        if offerings.index.size > 0:
            trade = offerings.iloc[0]
            buy_price = float(trade["bid"])
            size = round(float(balance/(buy_price*(1+fee))),6)
            buy = cbs.place_buy("BTC",buy_price,size)
            comet.store("orders",pd.DataFrame([buy]))
    comet.disconnect()
    count += 1
    print(count)
    time.sleep(3600)

1
2
3
4


In [ ]:
fls